In [1]:
from orderblockdetector import *
from binance.client import Client  # Assuming you have imported the Binance client
from binance.enums import HistoricalKlinesType


In [2]:

def get_historical_data(symbols, intervals, limit=180, save_path=None):
    """
    Retrieve historical data for a list of symbols from Binance API.
    """
    client = Client()  # Initialize the Binance client with your API credentials
    data = {}
    for symbol in symbols:
        # placeholder.write(f"Fetching data for {symbol}...")
        for inter in intervals:
            klines = client.get_historical_klines(symbol, inter, klines_type=HistoricalKlinesType.SPOT, limit=limit)
            df = pd.DataFrame(klines, columns=['datetime', 'open', 'high', 'low', 'close', 'volume',
                                                'close_time', 'qav', 'num_trades', 'taker_base_vol',
                                                'taker_quote_vol', 'ignore'])
            df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
            df.set_index('datetime', inplace=True)
            data[symbol] = df

            # Save the DataFrame to a pickle file if save_path is provided
            if save_path:
                df.to_pickle(f"{save_path}/{symbol}_{inter}_data.pkl")
    return data

In [3]:
# Fetch historical data only once.
selected_ticker = 'BTCUSDT'
symbols = ['BTCUSDT']
intervals = ['8h']
ob_length = 5

historical_data = get_historical_data(symbols, intervals, limit=200, save_path=None)
data = historical_data[selected_ticker].astype(float)
data['time'] = data.index

df_with_ob, active_bull_OB, active_bear_OB = detect_order_blocks(
    data, length=ob_length, bull_ext_last=3, bear_ext_last=3, mitigation='Wick'
)

In [5]:
active_bull_OB

[{'index': 145,
  'bull_top': 80641.60500000001,
  'bull_btm': 79058.0,
  'bull_avg': 79849.8025,
  'bull_left': Timestamp('2025-03-11 08:00:00'),
  'value': 79058.0}]

In [6]:
active_bear_OB

[{'index': 29,
  'bear_top': 105574.4,
  'bear_btm': 103567.2,
  'bear_avg': 104570.79999999999,
  'bear_left': Timestamp('2025-01-31 16:00:00'),
  'value': 105574.4},
 {'index': 49,
  'bear_top': 100137.99,
  'bear_btm': 98375.615,
  'bear_avg': 99256.8025,
  'bear_left': Timestamp('2025-02-07 08:00:00'),
  'value': 100137.99},
 {'index': 92,
  'bear_top': 98757.0,
  'bear_btm': 96814.475,
  'bear_avg': 97785.7375,
  'bear_left': Timestamp('2025-02-21 16:00:00'),
  'value': 98757.0},
 {'index': 119,
  'bear_top': 95000.0,
  'bear_btm': 91139.255,
  'bear_avg': 93069.6275,
  'bear_left': Timestamp('2025-03-02 16:00:00'),
  'value': 95000.0},
 {'index': 132,
  'bear_top': 91283.02,
  'bear_btm': 87975.025,
  'bear_avg': 89629.02249999999,
  'bear_left': Timestamp('2025-03-07 00:00:00'),
  'value': 91283.02},
 {'index': 184,
  'bear_top': 88765.43,
  'bear_btm': 87802.095,
  'bear_avg': 88283.7625,
  'bear_left': Timestamp('2025-03-24 08:00:00'),
  'value': 88765.43}]

In [4]:
df_with_ob

,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore,time,bull_ob,bear_ob,bull_mitigated,bear_mitigated
datetime,,,,,,,,,,,,,,,,
2025-01-22 00:00:00,106143.82,106394.46,105043.88,105084.77,5913.85108,1.737533e+12,6.251200e+08,947225.0,2673.52511,2.826445e+08,0.0,2025-01-22 00:00:00,NaN,NaN,False,False
2025-01-22 08:00:00,105084.77,105584.23,103662.58,104180.01,9015.86328,1.737562e+12,9.438010e+08,1695645.0,4165.81667,4.360779e+08,0.0,2025-01-22 08:00:00,NaN,NaN,False,False
2025-01-22 16:00:00,104180.01,104830.12,103339.12,103706.66,7318.97818,1.737590e+12,7.622237e+08,1721165.0,3662.25128,3.814509e+08,0.0,2025-01-22 16:00:00,NaN,NaN,False,False
2025-01-23 00:00:00,103706.66,103819.99,101575.50,102710.27,8882.46217,1.737619e+12,9.111254e+08,1257135.0,4165.54173,4.273681e+08,0.0,2025-01-23 00:00:00,NaN,NaN,False,False
2025-01-23 08:00:00,102710.28,106050.86,101262.28,105263.52,20454.98986,1.737648e+12,2.121490e+09,2510410.0,10372.86601,1.076228e+09,0.0,2025-01-23 08:00:00,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-28 00:00:00,87232.01,87515.67,85162.48,85212.99,7582.81673,1.743149e+12,6.534247e+08,961759.0,3136.88725,2.703736e+08,0.0,2025-03-28 00:00:00,NaN,NaN,False,False
2025-03-28 08:00:00,85212.98,85609.74,83953.86,84051.16,13333.39809,1.743178e+12,1.130567e+09,1355845.0,5470.65058,4.642029e+08,0.0,2025-03-28 08:00:00,NaN,NaN,False,False
2025-03-28 16:00:00,84051.15,84538.69,83585.00,84424.38,6266.51687,1.743206e+12,5.259549e+08,918481.0,2667.29953,2.239468e+08,0.0,2025-03-28 16:00:00,NaN,NaN,False,False


In [3]:
from tvDatafeed import TvDatafeed, Interval
tv = TvDatafeed()

historical_data = tv.get_hist(
    symbol="BTCUSDC",
    exchange="BINANCE",
    interval=Interval.in_4_hour,
    n_bars=360
)

In [9]:
historical_data

,symbol,open,high,low,close,volume
datetime,,,,,,
2025-04-14 19:00:00,BINANCE:BTCUSDC,84918.00,85795.99,83705.88,84097.98,1622.89989
2025-04-14 23:00:00,BINANCE:BTCUSDC,84094.00,85212.53,83856.00,84923.94,1096.02030
2025-04-15 03:00:00,BINANCE:BTCUSDC,84923.95,84973.56,84414.91,84589.56,349.31458
2025-04-15 07:00:00,BINANCE:BTCUSDC,84590.00,85399.00,84350.00,85366.62,437.68307
2025-04-15 11:00:00,BINANCE:BTCUSDC,85366.61,85856.42,85149.38,85718.68,520.35503
...,...,...,...,...,...,...
2025-06-12 23:00:00,BINANCE:BTCUSDC,107040.22,108476.00,106725.41,106847.99,855.65049
2025-06-13 03:00:00,BINANCE:BTCUSDC,106842.01,106999.99,105716.01,105718.01,882.55838
2025-06-13 07:00:00,BINANCE:BTCUSDC,105718.00,105736.27,102765.17,104363.99,1996.36650


In [24]:
import numpy as np

# Combine every two 4h rows into one 8h row, but ensure the resulting index always includes 07:00
historical_data_8h = historical_data.resample('8H', origin='07:00').agg({
    'symbol': 'first',
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
})



C:\Users\chaya\AppData\Local\Temp\ipykernel_20112\421322508.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  historical_data_8h = historical_data.resample('8H', origin='07:00').agg({


In [25]:
historical_data_8h

,symbol,open,high,low,close,volume
datetime,,,,,,
2025-04-14 15:00:00,BINANCE:BTCUSDC,84918.00,85795.99,83705.88,84097.98,1622.89989
2025-04-14 23:00:00,BINANCE:BTCUSDC,84094.00,85212.53,83856.00,84589.56,1445.33488
2025-04-15 07:00:00,BINANCE:BTCUSDC,84590.00,85856.42,84350.00,85718.68,958.03810
2025-04-15 15:00:00,BINANCE:BTCUSDC,85718.68,86471.17,84625.97,84908.23,2146.55207
2025-04-15 23:00:00,BINANCE:BTCUSDC,84913.99,85145.65,83601.00,83640.01,1304.45450
...,...,...,...,...,...,...
2025-06-12 07:00:00,BINANCE:BTCUSDC,108682.99,108854.00,107342.21,107648.01,799.86998
2025-06-12 15:00:00,BINANCE:BTCUSDC,107648.00,107870.75,106620.00,107040.22,1229.90880
2025-06-12 23:00:00,BINANCE:BTCUSDC,107040.22,108476.00,105716.01,105718.01,1738.20887
